In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

import tensorflow as tf
from keras import optimizers, Sequential
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from numpy.random import seed
seed(7)
tf.random.set_seed(11)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [ ]:
sign = lambda x: (1, -1)[x < 0]

def curve_shift(df, shift_by):
    '''
    This function will shift the binary labels in a dataframe.
    The curve shift will be with respect to the 1s.
    For example, if shift is -2, the following process
    will happen: if row n is labeled as 1, then
    - Make row (n+shift_by):(n+shift_by-1) = 1.
    - Remove row n.
    i.e. the labels will be shifted up to 2 rows up.

    Inputs:
    df       A pandas dataframe with a binary labeled column.
             This labeled column should be named as 'y'.
    shift_by An integer denoting the number of rows to shift.

    Output
    df       A dataframe with the binary labels shifted by shift.
    '''

    vector = df['Label'].copy()
    for s in range(abs(shift_by)):
        tmp = vector.shift(sign(shift_by))
        tmp = tmp.fillna(0)
        vector += tmp
    labelcol = 'Label'
    # Add vector to the df
    df.insert(loc=0, column=labelcol+'tmp', value=vector)
    # Remove the rows with labelcol == 1.
    df = df.drop(df[df[labelcol] == 1].index)
    # Drop labelcol and rename the tmp col as labelcol
    df = df.drop(labelcol, axis=1)
    df = df.rename(columns={labelcol+'tmp': labelcol})
    # Make the labelcol binary
    df.loc[df[labelcol] > 0, labelcol] = 1

    return df

In [ ]:
df=pd.read_csv('/content/24_hrs_fin.csv')
df

,TimeStamp,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Temp,Gas,RotX,RotY,RotZ,RotW,RotAcc,Pressure,HeartRate,Label,AQI
0,1670025600,-3128,2559,-415.0,4,8,-31.0,37.17,1620.0,0.408,0.625,0.070,0.662,3.142,1015.529,75,0,156
1,1670025601,-3212,2440,-519.0,0,0,-7.0,37.26,1565.0,0.399,0.634,0.064,0.659,3.142,1015.568,74,0,155
2,1670025602,-3256,2502,-541.0,-14,0,12.0,37.35,1662.0,0.398,0.636,0.063,0.658,3.142,1015.583,75,0,156
3,1670025603,-3219,2431,-564.0,-4,-11,8.0,37.42,1885.0,0.396,0.639,0.062,0.657,3.142,1015.583,73,0,157
4,1670025604,-3242,2451,-548.0,-4,0,6.0,37.45,2083.0,0.395,0.639,0.064,0.656,3.142,1015.560,72,0,161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86395,1670111995,-3315,2381,-570.0,-1,3,-7.0,37.49,2272.0,0.388,0.648,0.059,0.653,3.142,1015.646,73,0,158
86396,1670111996,-3286,2386,-581.0,2,7,-6.0,37.56,2235.0,0.388,0.648,0.057,0.653,3.142,1015.669,74,0,155
86397,1670111997,-3273,2372,-569.0,5,-19,23.0,37.65,2242.0,0.389,0.647,0.058,0.653,3.142,1015.654,75,0,159
86398,1670111998,-3270,2408,-566.0,0,7,-10.0,37.65,2729.0,0.391,0.645,0.058,0.654,3.142,1015.646,76,0,155


In [ ]:
df = df.drop(['TimeStamp'], axis=1)

In [ ]:
input_X = df.loc[:, df.columns != 'Label'].values  # converts the df to a numpy array
input_y = df['Label'].values
n_features = input_X.shape[1]  # number of features

In [ ]:
# Shift the response column y by 600 rows to do a 10-min ahead prediction.
df = curve_shift(df, shift_by = -600)

In [ ]:
df.to_csv('df.csv')

In [ ]:
def temporalize(X, y, lookback):
    '''
    Inputs
    X         A 2D numpy array ordered by time of shape:
              (n_observations x n_features)
    y         A 1D numpy array with indexes aligned with
              X, i.e. y[i] should correspond to X[i].
              Shape: n_observations.
    lookback  The window size to look back in the past
              records. Shape: a scalar.

    Output
    output_X  A 3D numpy array of shape:
              ((n_observations-lookback-1) x lookback x
              n_features)
    output_y  A 1D array of shape:
              (n_observations-lookback-1), aligned with X.
    '''
    output_X = []
    output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        output_y.append(y[i + lookback + 1])
    return np.squeeze(np.array(output_X)), np.array(output_y)

In [ ]:
X, y = temporalize(X = input_X, y = input_y, lookback = 200)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=DATA_SPLIT_PCT, random_state=SEED)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=DATA_SPLIT_PCT, random_state=SEED)

In [ ]:
X_train_y0 = X_train[y_train==0]
X_train_y1 = X_train[y_train==1]
X_valid_y0 = X_valid[y_valid==0]
X_valid_y1 = X_valid[y_valid==1]

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 200, n_features)
X_train_y0 = X_train_y0.reshape(X_train_y0.shape[0], 200, n_features)
X_train_y1 = X_train_y1.reshape(X_train_y1.shape[0], 200, n_features)
X_valid = X_valid.reshape(X_valid.shape[0], 200, n_features)
X_valid_y0 = X_valid_y0.reshape(X_valid_y0.shape[0], 200, n_features)
X_valid_y1 = X_valid_y1.reshape(X_valid_y1.shape[0], 200, n_features)
X_test = X_test.reshape(X_test.shape[0], 200, n_features)

In [ ]:
def flatten(X):
    '''
    Flatten a 3D array.

    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.

    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

def scale(X, scaler):
    '''
    Scale 3D array.

    Inputs
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    scaler       A scaler object, e.g., sklearn.preprocessing.StandardScaler, sklearn.preprocessing.normalize

    Output
    X            Scaled 3D array.
    '''
    for i in range(X.shape[0]):
        X[i, :, :] = scaler.transform(X[i, :, :])

    return X

In [ ]:
# Initialize a scaler using the training data.
scaler = StandardScaler().fit(flatten(X_train_y0))

In [ ]:
X_train_y0_scaled = scale(X_train_y0, scaler)

In [ ]:
a = flatten(X_train_y0_scaled)
print('colwise mean', np.mean(a, axis=0).round(6))
print('colwise variance', np.var(a, axis=0))

colwise mean [-0.  0.  0. -0. -0. -0.  0.  0. -0. -0.  0.  0. -0.  0.  0. -0.]
colwise variance [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
X_valid_scaled = scale(X_valid, scaler)
X_valid_y0_scaled = scale(X_valid_y0, scaler)
X_test_scaled = scale(X_test, scaler)

In [ ]:
timesteps =  X_train_y0_scaled.shape[1] # equal to the lookback
n_features =  X_train_y0_scaled.shape[2] # 59

epochs = 10
batch = 64
lr = 0.0001

In [ ]:
lstm_autoencoder = Sequential()
# Encoder
lstm_autoencoder.add(LSTM(32, activation='relu', input_shape=(timesteps, n_features), return_sequences=True))
lstm_autoencoder.add(LSTM(16, activation='relu', return_sequences=False))
lstm_autoencoder.add(RepeatVector(timesteps))
# Decoder
lstm_autoencoder.add(LSTM(16, activation='relu', return_sequences=True))
lstm_autoencoder.add(LSTM(32, activation='relu', return_sequences=True))
lstm_autoencoder.add(TimeDistributed(Dense(n_features)))

lstm_autoencoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 200, 32)           6272      
                                                                 
 lstm_1 (LSTM)               (None, 16)                3136      
                                                                 
 repeat_vector (RepeatVecto  (None, 200, 16)           0         
 r)                                                              
                                                                 
 lstm_2 (LSTM)               (None, 200, 16)           2112      
                                                                 
 lstm_3 (LSTM)               (None, 200, 32)           6272      
                                                                 
 time_distributed (TimeDist  (None, 200, 16)           528       
 ributed)                                               

In [ ]:
adam = optimizers.Adam(lr)
lstm_autoencoder.compile(loss='mse', optimizer=adam)

cp = ModelCheckpoint(filepath="lstm_autoencoder_classifier.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

lstm_autoencoder_history = lstm_autoencoder.fit(X_train_y0_scaled, X_train_y0_scaled,
                                                epochs=epochs,
                                                batch_size=batch,
                                                validation_data=(X_valid_y0_scaled, X_valid_y0_scaled),
                                                verbose=2).history

Epoch 1/10
828/828 - 291s - loss: nan - val_loss: nan - 291s/epoch - 352ms/step
Epoch 2/10
828/828 - 276s - loss: nan - val_loss: nan - 276s/epoch - 334ms/step
Epoch 3/10
828/828 - 283s - loss: nan - val_loss: nan - 283s/epoch - 342ms/step
Epoch 4/10
828/828 - 266s - loss: nan - val_loss: nan - 266s/epoch - 321ms/step
Epoch 5/10
828/828 - 266s - loss: nan - val_loss: nan - 266s/epoch - 322ms/step
Epoch 6/10
828/828 - 269s - loss: nan - val_loss: nan - 269s/epoch - 325ms/step
Epoch 7/10
828/828 - 280s - loss: nan - val_loss: nan - 280s/epoch - 339ms/step
Epoch 8/10
828/828 - 279s - loss: nan - val_loss: nan - 279s/epoch - 337ms/step
Epoch 9/10
828/828 - 274s - loss: nan - val_loss: nan - 274s/epoch - 331ms/step
Epoch 10/10
828/828 - 278s - loss: nan - val_loss: nan - 278s/epoch - 336ms/step


In [ ]:
plt.plot(lstm_autoencoder_history['loss'], linewidth=2, label='Train')
plt.plot(lstm_autoencoder_history['val_loss'], linewidth=2, label='Valid')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
valid_x_predictions = lstm_autoencoder.predict(X_valid_scaled)
mse = np.mean(np.power(flatten(X_valid_scaled) - flatten(valid_x_predictions), 2), axis=1)

error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': y_valid.tolist()})

precision_rt, recall_rt, threshold_rt = precision_recall_curve(error_df.True_class, error_df.Reconstruction_error)
plt.plot(threshold_rt, precision_rt[1:], label="Precision",linewidth=5)
plt.plot(threshold_rt, recall_rt[1:], label="Recall",linewidth=5)
plt.title('Precision and recall for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision/Recall')
plt.legend()
plt.show()

In [ ]:
test_x_predictions = lstm_autoencoder.predict(X_test_scaled)
mse = np.mean(np.power(flatten(X_test_scaled) - flatten(test_x_predictions), 2), axis=1)

error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': y_test.tolist()})

threshold_fixed = 0.3
groups = error_df.groupby('True_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Break" if name == 1 else "Normal")
ax.hlines(threshold_fixed, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();

In [ ]:
pred_y = [1 if e > threshold_fixed else 0 for e in error_df.Reconstruction_error.values]conf_matrix = confusion_matrix(error_df.True_class, pred_y)

plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
false_pos_rate, true_pos_rate, thresholds = roc_curve(error_df.True_class, error_df.Reconstruction_error)
roc_auc = auc(false_pos_rate, true_pos_rate,)

plt.plot(false_pos_rate, true_pos_rate, linewidth=5, label='AUC = %0.3f'% roc_auc)
plt.plot([0,1],[0,1], linewidth=5)

plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.legend(loc='lower right')
plt.title('Receiver operating characteristic curve (ROC)')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()